In [23]:
import pandas as pd
df_target = pd.read_csv('data/02_input_target.csv')
df_capacity = pd.read_csv('data/02_input_capacity.csv')
df_shipment_cost = pd.read_csv('data/02_03_input_shipmentsCost_example.csv')
df_production_cost = pd.read_csv('data/03_input_productionCost.csv')
df_shipment_cost

,Origin,Destination,Unit Cost
0,Japan,Australia,13.43
1,Japan,Brazil,10.42
2,Japan,Egypt,9.90
3,Japan,Italy,13.01
4,Japan,South Africa,8.92
5,Japan,France,5.60
6,Japan,India,6.75
7,Japan,United Kingdom,10.12
8,Japan,Russia,12.60
9,Australia,Brazil,10.07


In [24]:
import pandas as pd
from ortools.linear_solver import pywraplp

# Carica i dataset necessari
demand_df = df_target       # Country, Product, Month, Quantity
capacity_df = df_capacity     # Country, Monthly Capacity
shipment_cost_df = df_shipment_cost # Origin, Destination, Product, Cost_per_Unit
production_cost_df = df_production_cost # Country, Product, Cost_per_Unit

# Prepara dizionari
demand_df['Key'] = list(zip(demand_df['Country'], demand_df['Product'], demand_df['Month']))
demand = dict(zip(demand_df['Key'], demand_df['Quantity']))

capacity = dict(zip(capacity_df['Country'], capacity_df['Monthly Capacity']))

# Creiamo un dizionario per i costi di trasporto
transport_costs = {}
for _, row in shipment_cost_df.iterrows():
    key = (row['Origin'], row['Destination'])
    transport_costs[key] = row['Unit Cost']
    # key_inf = (row['Destination'], row['Origin'])
    # transport_costs[key] = np.inf

# Creiamo un dizionario per i costi di produzione
production_costs = {}
for _, row in production_cost_df.iterrows():
    key = (row['Country'], row['Product'])
    production_costs[key] = row['Unit Cost']

# Impianti e mesi
plants = list(capacity.keys())
months = sorted(demand_df['Month'].unique())
demand_keys = list(demand.keys())

# Inizializza il solver OR-Tools
solver = pywraplp.Solver.CreateSolver('SAT')

# Variabili decisionali: quanto produce ogni impianto per soddisfare domanda (i,j,p,m)
x = {}
for i in plants:
    for (j, p, m) in demand_keys:
        x[i, j, p, m] = solver.NumVar(0, solver.infinity(), f'x_{i}_{j}_{p}_{m}')

# Variabile per il massimo carico (da minimizzare)
max_load = solver.NumVar(0, 1, 'max_load')

# 1. Vincolo di domanda
for (j, p, m) in demand_keys:
    solver.Add(solver.Sum(x[i, j, p, m] for i in plants) == demand[(j, p, m)])

# 2. Vincoli di capacità impianto
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= capacity[i])

# 3. Vincoli per il carico massimo
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= max_load * capacity[i])

# 4. Funzione obiettivo: Minimizzare i costi di produzione e trasporto
total_cost = solver.Sum(
    x[i, j, p, m] * transport_costs.get((i, j, p), 0) # Costo di trasporto
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi i costi di produzione alla funzione obiettivo
total_cost += solver.Sum(
    x[i, j, p, m] * production_costs.get((i, p), 0)  # Costo di produzione
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi l'obiettivo (costo totale da minimizzare)
solver.Minimize(total_cost*1e-7 + max_load)

# Risoluzione
status = solver.Solve()

# Output della soluzione
if status == pywraplp.Solver.OPTIMAL:
    print(f"Status: Ottimale")
    print(f"Carico massimo usato: {max_load.solution_value():.2%}")
    print("Costi totali:", solver.Objective().Value())
    
    # Crea due liste separate per il piano di produzione e quello di spedizione
    production_data = []
    shipment_data = []

if status == pywraplp.Solver.OPTIMAL:
    print(f'Status: Ottimale\nCarico massimo usato: {max_load.solution_value():.2%}')
    for (i, j, p, m), var in x.items():
        if var.solution_value() > 0:
            print(f'{i} produce {var.solution_value():.0f} unità per {j} ({p}) a {m}')
else:
    print('Nessuna soluzione ottimale trovata.')


Status: Ottimale
Carico massimo usato: 100.00%
Costi totali: 8.533171729000014
Status: Ottimale
Carico massimo usato: 100.00%
Australia produce 1160 unità per Japan (BreatheEasy Inhalant) a Jan2004
Australia produce 1187 unità per Japan (BreatheEasy Inhalant) a Feb2004
Australia produce 1056 unità per Japan (BreatheEasy Inhalant) a Mar2004
Australia produce 1134 unità per Japan (BreatheEasy Inhalant) a Apr2004
Australia produce 1134 unità per Japan (BreatheEasy Inhalant) a May2004
Australia produce 1166 unità per Japan (BreatheEasy Inhalant) a Jun2004
Australia produce 1125 unità per Japan (BreatheEasy Inhalant) a Jul2004
Australia produce 1268 unità per Japan (BreatheEasy Inhalant) a Aug2004
Australia produce 1251 unità per Japan (BreatheEasy Inhalant) a Sep2004
Australia produce 1015 unità per Japan (BreatheEasy Inhalant) a Oct2004
Australia produce 1177 unità per Japan (BreatheEasy Inhalant) a Nov2004
Australia produce 1189 unità per Japan (BreatheEasy Inhalant) a Dec2004
Australia 

In [25]:
# initialize two dataframes for the solution
# production_plan = pd.DataFrame(columns=['Country', 'Product', 'Month', 'Quantity'])
# shipment_plan = pd.DataFrame(columns=['Origin', 'Destination', 'Product', 'Month', 'Quantity'])

shipment_data = []

# fill the production_plan dataframe with the results
for (i, j, p, m), var in x.items():
	shipment_data.append({
        'Origin': i,
		'Destination': j,
		'Product': p,
		'Month': m,
		'Quantity': int(var.solution_value())
	})
    
shipment_plan = pd.DataFrame(shipment_data)

In [26]:
shipment_plan[(~(shipment_plan['Origin'] == shipment_plan['Destination'])) & (shipment_plan['Quantity'] > 0)].reset_index(drop=True)

,Origin,Destination,Product,Month,Quantity
0,Australia,Japan,BreatheEasy Inhalant,Jan2004,1160
1,Australia,Japan,BreatheEasy Inhalant,Feb2004,1187
2,Australia,Japan,BreatheEasy Inhalant,Mar2004,1056
3,Australia,Japan,BreatheEasy Inhalant,Apr2004,1134
4,Australia,Japan,BreatheEasy Inhalant,May2004,1134
...,...,...,...,...,...
6651,United Kingdom,Russia,BrightBreeze Pet Odor Eliminator,Aug2004,821
6652,United Kingdom,Russia,BrightBreeze Pet Odor Eliminator,Sep2004,684
6653,United Kingdom,Russia,BrightBreeze Pet Odor Eliminator,Oct2004,623
6654,United Kingdom,Russia,BrightBreeze Pet Odor Eliminator,Nov2004,547


In [27]:
A = df_target.groupby(['Product']).agg({'Quantity': 'sum'}).reset_index()
B = shipment_plan.groupby(['Product']).agg({'Quantity': 'sum'}).reset_index()
A['Quantity'] - B['Quantity']

0     0
1     0
2     0
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    0
Name: Quantity, Length: 100, dtype: int64

In [28]:
production_plan = shipment_plan.groupby(['Origin', 'Product', 'Month'], as_index=False).agg({'Quantity': 'sum'})
production_plan.rename(columns={'Origin': 'Country'}, inplace=True)
production_plan.to_csv('outputs/03_output_productionPlan_1239.csv', index=False)
shipment_plan.to_csv('outputs/03_output_shipments_1239.csv', index=False)

# Without symmetric shipment

In [82]:
import numpy as np
cost_matrix = df_shipment_cost.pivot_table(index="Origin", columns="Destination", values="Unit Cost")

# Optional: Fill NaN with a value if needed, like 0 or np.inf
# cost_matrix = cost_matrix.fillna(0)
cost_matrix.loc["Russia"] = pd.NA
# insert the Japan column and fill with NA
cost_matrix.loc[:, "Japan"] = pd.NA
# Rearange the rows in the order of the columns
cost_matrix = cost_matrix.reindex(columns=cost_matrix.columns, index=cost_matrix.index)
# Fill the NAN with infinity
# cost_matrix = cost_matrix.fillna(float(np.inf))
# Fill Nan with symmetric values
cost_matrix = cost_matrix.fillna(cost_matrix.T)
#print(cost_matrix.columns.values)
# print(cost_matrix)
# Fill the elements with same origin and destination by name with zero
for i in cost_matrix.columns.values:
	cost_matrix.loc[i, i] = 0
#print(cost_matrix)

# Rearange to a single dataframe
cost_matrix = cost_matrix.stack().reset_index()
cost_matrix.columns = ['Origin', 'Destination', 'Unit Cost']
cost_matrix.head(20)

/tmp/ipykernel_38991/324858150.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cost_matrix.loc["Russia"] = pd.NA


,Origin,Destination,Unit Cost
0,Australia,Australia,0
1,Australia,Brazil,10.07
2,Australia,Egypt,8.89
3,Australia,France,10.61
4,Australia,India,7.78
5,Australia,Italy,8.35
6,Australia,Russia,11.06
7,Australia,South Africa,13.61
8,Australia,United Kingdom,9.33
9,Australia,Japan,13.43


In [102]:
import pandas as pd
from ortools.linear_solver import pywraplp

# Carica i dataset necessari
demand_df = df_target       # Country, Product, Month, Quantity
capacity_df = df_capacity     # Country, Monthly Capacity
shipment_cost_df = df_shipment_cost 
# shipment_cost_df = cost_matrix # Origin, Destination, Cost_per_Unit
production_cost_df = df_production_cost # Country, Product, Cost_per_Unit

# Prepara dizionari
demand_df['Key'] = list(zip(demand_df['Country'], demand_df['Product'], demand_df['Month']))
demand = dict(zip(demand_df['Key'], demand_df['Quantity']))

capacity = dict(zip(capacity_df['Country'], capacity_df['Monthly Capacity']))

# Creiamo un dizionario per i costi di trasporto
transport_costs = {}
for _, row in shipment_cost_df.iterrows():
    key = (row['Origin'], row['Destination'])
    transport_costs[key] = row['Unit Cost']
    # key_inf = (row['Destination'], row['Origin'])
    # transport_costs[key] = np.inf
print(transport_costs)

# Creiamo un dizionario per i costi di produzione
production_costs = {}
for _, row in production_cost_df.iterrows():
    key = (row['Country'], row['Product'])
    production_costs[key] = row['Unit Cost']

# Impianti e mesi
plants = list(capacity.keys())
months = sorted(demand_df['Month'].unique())
demand_keys = list(demand.keys())

# Inizializza il solver OR-Tools
solver = pywraplp.Solver.CreateSolver('SAT')

# Variabili decisionali: quanto produce ogni impianto per soddisfare domanda (i,j,p,m)
x = {}
for i in plants:
    for (j, p, m) in demand_keys:
        x[i, j, p, m] = solver.NumVar(0, solver.infinity(), f'x_{i}_{j}_{p}_{m}')

# Variabile per il massimo carico (da minimizzare)
max_load = solver.NumVar(0, 1, 'max_load')

# 1. Vincolo di domanda
for (j, p, m) in demand_keys:
    solver.Add(solver.Sum(x[i, j, p, m] for i in plants) == demand[(j, p, m)])

# 2. Vincoli di capacità impianto
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= capacity[i])

# 3. Vincoli per il carico massimo
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= max_load * capacity[i])

# 4. Funzione obiettivo: Minimizzare i costi di produzione e trasporto
total_cost = solver.Sum(
    # x[i, j, p, m] * transport_costs.get((i, j), 0) # Costo di trasporto se lo trova, sennò 0
    # x[i, j, p, m] * transport_costs.get((i, j), np.inf) # Costo di trasporto se lo trova, sennò infinito
    x[i, j, p, m] * transport_costs.get((i, j), transport_costs.get((j,i))) if i !=j else 0 # Costo di trasporto se lo trova, sennò simmetrico
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi i costi di produzione alla funzione obiettivo
total_cost += solver.Sum(
    x[i, j, p, m] * production_costs.get((i, p), 0)  # Costo di produzione se lo trova, sennò 0
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi l'obiettivo (costo totale da minimizzare)
solver.Minimize(total_cost*1e-8 + max_load)

# Risoluzione
status = solver.Solve()

# Output della soluzione
if status == pywraplp.Solver.OPTIMAL:
    print(f"Status: Ottimale")
    print(f"Carico massimo usato: {max_load.solution_value():.2%}")
    print("Costi totali:", solver.Objective().Value())
    
    # Crea due liste separate per il piano di produzione e quello di spedizione
    production_data = []
    shipment_data = []

if status == pywraplp.Solver.OPTIMAL:
    print(f'Status: Ottimale\nCarico massimo usato: {max_load.solution_value():.2%}')
    for (i, j, p, m), var in x.items():
        if var.solution_value() > 0:
            print(f'{i} produce {var.solution_value():.0f} unità per {j} ({p}) a {m}')
else:
    print('Nessuna soluzione ottimale trovata.')


{('Japan', 'Australia'): 13.43, ('Japan', 'Brazil'): 10.42, ('Japan', 'Egypt'): 9.9, ('Japan', 'Italy'): 13.01, ('Japan', 'South Africa'): 8.92, ('Japan', 'France'): 5.6, ('Japan', 'India'): 6.75, ('Japan', 'United Kingdom'): 10.12, ('Japan', 'Russia'): 12.6, ('Australia', 'Brazil'): 10.07, ('Australia', 'Egypt'): 8.89, ('Australia', 'Italy'): 8.35, ('Australia', 'South Africa'): 13.61, ('Australia', 'France'): 10.61, ('Australia', 'India'): 7.78, ('Australia', 'United Kingdom'): 9.33, ('Australia', 'Russia'): 11.06, ('Brazil', 'Egypt'): 10.35, ('Brazil', 'Italy'): 8.28, ('Brazil', 'South Africa'): 10.39, ('Brazil', 'France'): 9.15, ('Brazil', 'India'): 10.28, ('Brazil', 'United Kingdom'): 7.3, ('Brazil', 'Russia'): 11.9, ('Egypt', 'Italy'): 10.86, ('Egypt', 'South Africa'): 9.72, ('Egypt', 'France'): 10.98, ('Egypt', 'India'): 11.43, ('Egypt', 'United Kingdom'): 8.9, ('Egypt', 'Russia'): 8.51, ('Italy', 'South Africa'): 12.76, ('Italy', 'France'): 8.42, ('Italy', 'India'): 10.82, ('It

In [103]:
# initialize two dataframes for the solution
# production_plan = pd.DataFrame(columns=['Country', 'Product', 'Month', 'Quantity'])
# shipment_plan = pd.DataFrame(columns=['Origin', 'Destination', 'Product', 'Month', 'Quantity'])

shipment_data = []

# fill the production_plan dataframe with the results
for (i, j, p, m), var in x.items():
	shipment_data.append({
        'Origin': i,
		'Destination': j,
		'Product': p,
		'Month': m,
		'Quantity': int(var.solution_value())
	})
    
shipment_plan = pd.DataFrame(shipment_data)

In [104]:
production_plan = shipment_plan.groupby(['Origin', 'Product', 'Month'], as_index=False).agg({'Quantity': 'sum'})
production_plan.rename(columns={'Origin': 'Country'}, inplace=True)
production_plan.to_csv('outputs/03_output_productionPlan_1239.csv', index=False)
shipment_plan.to_csv('outputs/03_output_shipments_1239.csv', index=False)